In [1]:
#encoding utf-8
import numpy as np
import pandas as pd
import re
import itertools
from IPython.display import Image

In [2]:
DTFQA = pd.read_pickle('../01_Chat_Identfier/db/DTFQA_v03')
print len(DTFQA)
DTFQA.iloc[0:5,:]

130


,No.,utterances,intent,entity,nlp_class,nlp_name,term,pos
0,1,請問洗手間在哪?,問門市,"分店::內部資訊,洗手間;",NLP_Major_05,店內資訊,"[請問, 洗手間, 在哪, ?]","[qhead, branchinnerinfob, where, x]"
1,1,你們化妝室怎麼走?,問門市,"分店::內部資訊,化妝室;",NLP_Major_05,店內資訊,"[你們, 化妝室, 怎麼走, ?]","[nh, branchinnerinfob, where, x]"
2,1,我想上廁所,問門市,"分店::內部資訊,廁所;",NLP_Major_05,店內資訊,"[我, 想, 上, 廁所]","[r, v, f, branchinnerinfob]"
3,3,什麼時候不需要排隊?,問門市,"分店::候位,排隊;",NLP_Major_05,店內資訊,"[什麼時候, 不需要, 排隊, ?]","[when, ineg, branchinnerinfoa, x]"
4,3,哪時候人最少,問門市,"分店::候位,人最少;",NLP_Major_05,店內資訊,"[哪時候, 人最少]","[when, branchinnerinfoa]"


In [3]:
DTFQA_STATISTIC = pd.read_pickle('../01_Chat_Identfier/db/DTFQA_STATISTIC')
print len(DTFQA_STATISTIC)
DTFQA_STATISTIC.iloc[0:5,:]

931


,pos,term
0,qhead,請問
1,branchinnerinfob,洗手間
2,where,在哪
3,x,?
4,nh,你們


In [4]:
#取得所有 term 的所有可能 (移除重複)
POS_DETAIL = pd.DataFrame({'count' : DTFQA_STATISTIC.groupby( [ "pos", "term"] ).size()}).reset_index()
POS_DETAIL.iloc[0:10,:]
print 'all kind of term == ' + str(len(POS_DETAIL))

all kind of term == 256


In [5]:
POS_DETAIL.iloc[0:10,:]

,pos,term,count
0,a,一般,2
1,a,好,3
2,an,安全,2
3,branchinnerinfoa,人最少,1
4,branchinnerinfoa,候位,1
5,branchinnerinfoa,排隊,1
6,branchinnerinfob,化妝室,1
7,branchinnerinfob,廁所,1
8,branchinnerinfob,洗手間,1
9,c,以外,1


In [6]:
def check_pos_pair_in_DTFQA(p1,p2):
    ret = 'FALSE'
    for idx in range(len(DTFQA)):
        fork_sentence_term = list(np.array(DTFQA['term'][idx]))
        fork_sentence_pos  = list(np.array(DTFQA['pos'][idx]))
        #print repr(DTFQA['term'][idx]).decode('unicode-escape')
        
        if p1 in fork_sentence_pos:
            p1_index = fork_sentence_pos.index(p1)
            fork_sentence_pos.remove(p1)
            del fork_sentence_term[p1_index]
            
            #print 'match pos'
            #print repr(fork_sentence_pos).decode('unicode-escape')
            #print repr(fork_sentence_term).decode('unicode-escape')
        if p2 in fork_sentence_term:
            fork_sentence_term.remove(p2)
            #print 'match term'
            #print repr(fork_sentence_term).decode('unicode-escape')
        if len(DTFQA['term'][idx]) == len(fork_sentence_term) + 2:
            print repr(DTFQA['term'][idx]).decode('unicode-escape')
            #print 'orig_len = ' + str(len(DTFQA['term'][idx])) + ', after removement =' + str(len(fork_sentence_term))
            return 'TRUE'
    return ret      

In [9]:
p1 = u'dtfappetizer'
p2 = u'口味'
ret = check_pos_pair_in_DTFQA(p1,p2)
print ret

[u'鼎泰豐', u'的', u'泡菜', u'和', u'一般', u'市售', u'的', u'泡菜', u'口味', u'不同', u'，', u'請問', u'是', u'屬', u'於', u'何種', u'口味', u'?']
TRUE


In [8]:
pd.set_option("display.max_rows",100)
# DTF要處理的 n 集合
idx = np.array([bool(re.search(u'branch.*|^n$|dtf.*|ingredient', element)) for element in POS_DETAIL['pos']])
#idx = np.array([bool(re.search(u'^n$', element)) for element in POS_DETAIL['pos']])
print len(POS_DETAIL.iloc[idx,:])
DTF_N_SET = POS_DETAIL.iloc[idx,0:2]
DTF_N_SET

62


,pos,term
3,branchinnerinfoa,人最少
4,branchinnerinfoa,候位
5,branchinnerinfoa,排隊
6,branchinnerinfob,化妝室
7,branchinnerinfob,廁所
8,branchinnerinfob,洗手間
44,dtfappetizer,泡菜
45,dtfbranch,信義店
46,dtfbranchkinda,其他店
47,dtfbranchkinda,台灣門市


In [10]:
# all kind of (N,N) pair, 不只是同一句有出現
# 62*62 = 3844 --> 1020
ALL_NN_PAIR = list(set(list(itertools.product(DTF_N_SET['pos'], DTF_N_SET['term']))))
DTF_NN_PAIR = []
print len(ALL_NN_PAIR)
for (p1, p2) in ALL_NN_PAIR: 
    #print repr(p1).decode('unicode-escape') + ',' + repr(p2).decode('unicode-escape')
    ret = check_pos_pair_in_DTFQA(p1,p2)
    if ret == 'TRUE':
        print repr(p1).decode('unicode-escape') + ',' + repr(p2).decode('unicode-escape')
        DTF_NN_PAIR.append([p1,p2])
print len(DTF_NN_PAIR)


1020
[u'鼎泰豐', u'的', u'酸辣湯', u'加', u'了', u'哪些', u'東西']
u'dtfsoup',u'鼎泰豐'
[u'鼎泰豐', u'賣', u'的', u'是', u'港式點心', u'嗎', u'?']
u'dtfbranchkindb',u'鼎泰豐'
[u'請問', u'那些', u'東西', u'沒有', u'豬肉']
u'ingredient',u'豬肉'
[u'店內', u'有什麼', u'是', u'素食', u'的', u'食物', u'?']
u'dtffoodkind',u'店內'
[u'鼎泰豐', u'的', u'泡菜', u'和', u'一般', u'市售', u'的', u'泡菜', u'口味', u'不同', u'，', u'請問', u'是', u'屬', u'於', u'何種', u'口味', u'?']
u'dtfbrand',u'泡菜'
[u'你們', u'的', u'肉類', u'有', u'驗過', u'瘦肉精', u'嗎']
u'dtffoodsaftya',u'瘦肉精'
[u'我', u'今天', u'不想', u'吃肉', u'，', u'哪些', u'是', u'素的']
u'dtffoodkind',u'素的'
[u'鼎泰豐', u'的', u'泡菜', u'和', u'一般', u'市售', u'的', u'泡菜', u'口味', u'不同', u'，', u'請問', u'是', u'屬', u'於', u'何種', u'口味', u'?']
u'dtfappetizer',u'鼎泰豐'
[u'鼎泰豐', u'的', u'東西', u'有', u'瘦肉精', u'嗎', u'?']
u'dtffoodsaftyb',u'東西'
[u'因為', u'宗教', u'因素', u'不吃', u'豬肉', u'，', u'請問', u'蒸籠類', u'鹹', u'的', u'哪些', u'不含', u'豬肉', u'？']
u'dtfingredient',u'宗教'
[u'你們', u'的', u'東西', u'安全', u'嗎', u'?', u' ', u'有', u'驗過', u'嗎', u'?']
u'ingredient',u'驗過'
[u'鼎泰豐', u'的', u'食材',

In [ ]:
# SAVE TO dataframe
NN_NUMPY=np.array(DTF_NN_PAIR)
NN_POS_PAIR = pd.DataFrame(NN_NUMPY)
NN_POS_PAIR.columns = ['NPOS','NTERM']
NN_POS_PAIR.to_pickle('./NN_POS_PAIR')

In [ ]:
# DTF要處理的 v 集合
idx = np.array([bool(re.search(u'^v$|^vg$', element)) for element in POS_DETAIL['pos']])
print len(POS_DETAIL.iloc[idx,:])
DTF_V_SET = POS_DETAIL.iloc[idx,:]
DTF_V_SET


In [ ]:
# all kind of (V,N) pair, 不只是同一句有出現
# 62 * 34 = 2108 -> 578
#ALL_NV_PAIR = list(set(list(itertools.product(DTF_N_SET['pos'], DTF_V_SET['term']))))
# 17 * 34 = 2108 -> 578
ALL_NV_PAIR = list(itertools.product(set(DTF_N_SET['pos']), DTF_V_SET['term']))
DTF_NV_PAIR = []
print len(list(ALL_NV_PAIR))
for (p1, p2) in ALL_NV_PAIR: 
    #print repr(p1).decode('unicode-escape') + ',' + repr(p2).decode('unicode-escape')
    ret = check_pos_pair_in_DTFQA(p1,p2)
    if ret == 'TRUE':
        print repr(p1).decode('unicode-escape') + ',' + repr(p2).decode('unicode-escape')
        DTF_NV_PAIR.append([p1,p2])
#print repr(DTF_NV_PAIR).decode('unicode-escape')

In [ ]:
# SAVE TO dataframe
NV_NUMPY=np.array(DTF_NV_PAIR)
NV_POS_PAIR = pd.DataFrame(NV_NUMPY)
NV_POS_PAIR.columns = ['NPOS','VTERM']
NV_POS_PAIR.to_pickle('./NV_POS_PAIR')

In [ ]:
#all kind of (A,N) pair

In [ ]:
#==================================================================================================
# 單獨測試 pos pair
#==================================================================================================

In [ ]:
len(set(DTF_N_SET['pos']))

In [ ]:
# Query for specific none on DTF_NV_PAIR
# 取德特定組合內容, 下面的例子是找 dtfsoup 相關所有的 v 的組合
# 也可以翻譯成: 如果出現了 dtfsoup 這個名詞, 整句話可以出現的動詞只能是下面的幾個case
# 當然還要考慮 w2v 的 v, 但不是在這個時候加入
NV_NUMPY=np.array(DTF_NV_PAIR)
idx = np.where(NV_NUMPY[:,0] == u'dtfbranch')
print repr(NV_NUMPY[idx,:]).decode('unicode-escape')
NV_POS_PAIR = pd.DataFrame(NV_NUMPY)
NV_POS_PAIR.columns = ['NPOS','VTERM']


In [11]:
# Query for specific none on DTF_NN_PAIR
# 取德特定組合內容, 下面的例子是找 dtfsoup 相關所有的 v 的組合
# 也可以翻譯成: 如果出現了 dtfsoup 這個名詞, 整句話可以出現的動詞只能是下面的幾個case
# 當然還要考慮 w2v 的 v, 但不是在這個時候加入
NN_NUMPY=np.array(DTF_NN_PAIR)
idx = np.where(NN_NUMPY[:,0] == u'dtfxiaolongbao')
print repr(NN_NUMPY[idx,:]).decode('unicode-escape')
NN_POS_PAIR = pd.DataFrame(NN_NUMPY)
NN_POS_PAIR.columns = ['NPOS','NTERM']

array([[[u'dtfxiaolongbao', u'面包'],
        [u'dtfxiaolongbao', u'小籠包'],
        [u'dtfxiaolongbao', u'有什麼']]], 
      dtype='<U14')


In [ ]:
NN_POS_PAIR

In [ ]:
ALL_VN_PAIR = itertools.product(DTF_V_SET['term'], DTF_N_SET['term'])
list(ALL_VN_PAIR)

In [ ]:
#print repr(DTF_VN_PAIR[10]).decode('unicode-escape')
print 'length DTV_V_SET   = ' + str(len(DTF_V_SET))
print 'length ALL_VN_PAIR = ' + str(len(ALL_VN_PAIR))
print 'length DTF_VN_PAIR = ' + str(len(DTF_VN_PAIR))

In [ ]:
DTF_N_PAIR[0]
if [u'泡菜',u'泡菜'] in DTF_N_PAIR:
    print 'yes'
else:
    print 'no'

In [ ]:
#==================================================================================================
# 以下為測試code
#==================================================================================================

In [ ]:
def check_pos_pair_in_DTFQA(p1,p2):
    ret = 'FALSE'
    for sentence in DTFQA['term']:
        fork_sentence = list(np.array(sentence))
        if p1 in fork_sentence:
            fork_sentence.remove(p1)
        if p2 in fork_sentence_term:
            fork_sentence.remove(p2)  
        if len(sentence) == len(fork_sentence) + 2:
            print repr(sentence).decode('unicode-escape')
            print 'orig_len = ' + str(len(sentence)) + ', after removement =' + str(len(sentence))
            return 'TRUE'
    return ret      


In [ ]:
sentence = list(np.array(DTFQA['term'][4]))
#print repr(sentence).decode('unicode-escape')
sentence.append(u'哪時候')
print repr(sentence).decode('unicode-escape')


In [ ]:
p1 = u'哪時候'
p2 = u'人'
if p1 in sentence:
    sentence.remove(p1)
if p2 in sentence:
    sentence.remove(p2)
print repr(sentence).decode('unicode-escape')

In [ ]:
print repr(DTFQA['term'][4]).decode('unicode-escape')

In [ ]:
target_idx = np.where(POS_DETAIL['pos'] == 'n')
POS_DETAIL.iloc[target_idx[0],:]
print len(POS_DETAIL.iloc[target_idx[0],:])

In [ ]:
#np.any([p1*3],DTFTERM0)
print (np.array(DTFTERM4) == p1)

In [ ]:
#print repr(DTFTERM
#print repr(DTFQA['term'][4]).decode('unicode-escape')
DTFTERM0 = DTFQA['term'][0]
DTFTERM4 = DTFQA['term'][4]
print repr(DTFTERM0).decode('unicode-escape')
print repr(DTFTERM4).decode('unicode-escape')

In [ ]:
#print np.array(list(map(lambda v: re.sub(r'dtf','^n$', v) ,POS_DETAIL['pos'])))
#type(POS_DETAIL['pos'])
idx = np.where((POS_DETAIL['pos'] == u'branchinnerinfoa') | (POS_DETAIL['pos'] == 'n'))
POS_DETAIL.iloc[idx[0],:]

In [ ]:
idx = np.array([bool(re.search(u'food', element)) for element in POS_DETAIL['pos']])
print len(POS_DETAIL.iloc[idx,:])
POS_DETAIL.iloc[idx,:]

In [ ]:
pd.set_option("display.max_rows",100)
target_idx = np.where(POS_DETAIL['pos'] == 'n')
POS_DETAIL.iloc[target_idx[0],:]
N_LIST = POS_DETAIL.iloc[target_idx[0],:]
N_LIST

In [ ]:
#all kind of (N,N) pair

pos_list  = []
term_list = []
for idx in range(len(DTFQA['pos'])):
    pos_list  += DTFQA['pos'][idx]
    term_list += DTFQA['term'][idx]

In [ ]:
# 數字有用,文字不行
# create all pair combination in DTFQA.[pos]
# http://stackoverflow.com/questions/1208118/using-numpy-to-build-an-array-of-all-combinations-of-two-arrays
# cartesian
np.array(np.meshgrid([1, 2, 3],[1, 2, 3])).T.reshape(-1,2)
np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7])).T.reshape(-1,3)


In [ ]:
# cartesian 這個方法不知道怎麼 run
def cartesian2(arrays):
    arrays = [np.asarray(a) for a in arrays]
    shape = (len(x) for x in arrays)

    ix = np.indices(shape, dtype=int)
    ix = ix.reshape(len(arrays), -1).T

    for n, arr in enumerate(arrays):
        ix[:, n] = arrays[n][ix[:, n]]

    return ix

In [ ]:
# cartesian 這個方法數字有用,文字不行
def cartesian_coord(*arrays):
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    return points

a = np.array([1,2])
print(cartesian_coord(*2*[a]))

In [ ]:
# cartesian 這個方法文字數字都可以
import itertools
#a_p1 = np.arange(0, 4, 1)
#a_p2 = np.arange(20, 25, 1)
a_p1 = np.array(['a','b'])
a_p2 = np.array(['c','d'])
params = itertools.product(a_p1, a_p2)
for (p1, p2) in params:
    print(p1, p2)

In [ ]:
DTFQA['term']

In [ ]:
params = itertools.product(DTF_N_SET['term'], DTF_N_SET['term'])
for (p1, p2) in params:
    print repr(p1).decode('unicode-escape') + ',' + repr(p2).decode('unicode-escape')

In [ ]:
aList = [123, 'xyz', 'zara', 'abc', 'xyz'];

aList.remove('xyz');
print "List : ", aList
aList.remove('abc');
print "List : ", aList

In [ ]:
for idx in range(len(DTFQA)):
    sentence_term = DTFQA['term'][idx]
    print sentence_term                 
                  
    #print sentence_pos

In [ ]:
DTFQA['term'][0]